# Custom COMPLETE function
Wrap a SQL call in a fucntion that allows end user to specify temp and max tokens.

In [ ]:
def my_complete(model, context, temp = 0, max_tokens: int = 18000):
    sql = F"""SELECT SNOWFLAKE.CORTEX.COMPLETE(
            '{model}',
            [
                {{
                    'role': 'user',
                    'content': '{context}'
                }}
            ],
            {{
                'max_tokens': {max_tokens}, 
                'temperature' : {temp} 
            }}
        ) as inference;"""
    inference_raw = session.sql(sql).to_pandas().loc[0,"INFERENCE"]
    inference_json = json.loads(inference_raw)
    inference_raw = inference_json['choices'][0]['messages']
    return inference_raw

# Synthetic Data Prompts
Create a few prompts to generate the syntehtic data we will build against

In [ ]:
#this prompt creates the table AND creates some sample Inserts.
def generate_data_prompt(): 
    prompt= f"""
    <role>
    You are an experienced dev ops professional deeply knowledgeable on computer systems that support a very large company. You are an expert at the metadata that is captured about change requests.
    A change request is a formal proposal for an alteration to the computer system that you manage.
    As a dev ops expert, you specialize in creating the detailed information that is captured about a change request.
    </role>

    <task>: Follow these instructions,
    1) Considering the data_specs and your <role> create data definition language (ddl) to create a table with the appropriate columns. Then, 
    2) Considering the data_specs section and your <role> create data definition language (ddl) to fill the table with synthetic data set about change requests. 
    </task>
    
    <data_specs> 
    The metadata for the data is below and follows this format: (column_name: data_type : description : sample_values):
    
    Description: VARCHAR(16777216) : A summary of the change : ''Upgrade to the existing DataSync API from version 3.2 to 3.4 in the Production environment. The update includes several key performance optimizations, enhanced security features, and bug fixes that address issues with data consistency and processing time.
    The major components of this change include:
    
    API Version Update: Migrating from DataSync API v3.2 to v3.4 to support faster data ingestion and processing.
    Security Enhancements: Implementation of OAuth 2.0-based authentication to replace the legacy basic authentication mechanism, improving overall security for API transactions.
    Error Handling: Enhanced error codes and more descriptive responses for improved debugging in the event of failure.
    Database Schema Update: Modifications to the backend MySQL database to accommodate new data types introduced in version 3.4.
    Testing will be performed in the staging environment (v3.4-Stage) before deployment to ensure backwards compatibility with existing systems. No downtime is expected during the deployment, but a rollback plan has been prepared in case of critical issues.'', 
       
    Date: DATE : The date the request was made : 2025-02-03
    
    Impact: VARCHAR(4000) : How the change will affect the project, including cost, quality, risk, scope, duration, and schedule 
    
    Priority: VARCHAR(400) : How quickly the change should be approved and implemented : ''immediately'', ''within release window'', ''after approval''
    
    Risk: VARCHAR(400) : The risk level of the change as described by development team : ''low risk'', ''moderate risk'', ''high risk''
    
    Justification: VARCHAR(4000) : The reason for the change : ''preventative maintenance'', ''patch'', ''planned release as part of project koala''
    
    State: VARCHAR(400) : The status of the change request : ''new'', ''under review'', ''approved'', ''deferred'', ''rejected'' 
    
    Disposition: VARCHAR(400) : An explanation for approved, deferred, or rejected changes : ''no peer review'', ''manager override'', ''uniform agreement''
    
    Category: VARCHAR(400) : The category of the change : ''planned'', ''unplanned'', ''emergency'' 
    
    Change number: VARCHAR(100) : A unique ID for tracking the request :''AB1672'', ''723CS'', ''6D62EE''
    
    </data_specs>

    <output>
    Produce a single create table statement and 3 INSERT statements placing data into the table.
    Absolutely only return the sql and no other wording.
    </output>
    """
    return prompt

#this prompt ONLY generates Insert Statements
def generate_more_prompt(rows:int): 
    prompt= f"""
    <role>
        You are an experienced dev ops professional deeply knowledgeable on computer systems that support a very large company. You are an expert at the metadata that is captured about change requests.
        A change request is a formal proposal for an alteration to the computer system that you manage.
        As a dev ops expert, you specialize in creating the detailed information that is captured about a change request.
    </role>

    <data_specs> 
        The metadata for the data is below and follows this format: (column_name: data_type : description : sample_values):
        
        Description: VARCHAR(16777216) : A summary of the change : ''Upgrade to the existing DataSync API from version 3.2 to 3.4 in the Production environment. The update includes several key performance optimizations, enhanced security features, and bug fixes that address issues with data consistency and processing time.
        The major components of this change include:
        
        API Version Update: Migrating from DataSync API v3.2 to v3.4 to support faster data ingestion and processing.
        Security Enhancements: Implementation of OAuth 2.0-based authentication to replace the legacy basic authentication mechanism, improving overall security for API transactions.
        Error Handling: Enhanced error codes and more descriptive responses for improved debugging in the event of failure.
        Database Schema Update: Modifications to the backend MySQL database to accommodate new data types introduced in version 3.4.
        Testing will be performed in the staging environment (v3.4-Stage) before deployment to ensure backwards compatibility with existing systems. No downtime is expected during the deployment, but a rollback plan has been prepared in case of critical issues.'', 
           
        Date: DATE : The date the request was made : 2025-02-03
        
        Impact: VARCHAR(4000) : How the change will affect the project, including cost, quality, risk, scope, duration, and schedule 
        
        Priority: VARCHAR(400) : How quickly the change should be approved and implemented : ''immediately'', ''within release window'', ''after approval''
        
        Risk: VARCHAR(400) : The risk level of the change as described by development team : ''low risk'', ''moderate risk'', ''high risk''
        
        Justification: VARCHAR(4000) : The reason for the change : ''preventative maintenance'', ''patch'', ''planned release as part of project koala''
        
        State: VARCHAR(400) : The status of the change request : ''new'', ''under review'', ''approved'', ''deferred'', ''rejected'' 
        
        Disposition: VARCHAR(400) : An explanation for approved, deferred, or rejected changes : ''no peer review'', ''manager override'', ''uniform agreement''
        
        Category: VARCHAR(400) : The category of the change : ''planned'', ''unplanned'', ''emergency'' 
        
        Change number: VARCHAR(100) : A unique ID for tracking the request :''AB1672'', ''723CS'', ''6D62EE''
    
    </data_specs>

    <sample_insert_statement>
        INSERT INTO GEN_AI_FSI.DTCC_HACKATHON.CHANGE_REQUEST_RAW (ChangeNumber, Description, Date, Impact, Priority, Risk, Justification, State, Disposition, Category) VALUES
        (''AB1672'', ''This change request proposes an upgrade to the existing DataSync API from version 3.2 to 3.4 in the Production environment. The update includes several key performance optimizations, enhanced security features, and bug fixes that address issues with data consistency and processing time. The major components of this change include: API Version Update: Migrating from DataSync API v3.2 to v3.4 to support faster data ingestion and processing. Security Enhancements: Implementation of OAuth 2.0-based authentication to replace the legacy basic authentication mechanism, improving overall security for API transactions. Error Handling: Enhanced error codes and more descriptive responses for improved debugging in the event of failure. Database Schema Update: Modifications to the backend MySQL database to accommodate new data types introduced in version 3.4. Testing will be performed in the staging environment (v3.4-Stage) before deployment to ensure backwards compatibility with existing systems. No downtime is expected during the deployment, but a rollback plan has been prepared in case of critical issues.'', ''2024-02-03'', ''Improved performance and security with minimal downtime'', ''immediately'', ''low risk'', ''preventative maintenance'', ''new'', ''no peer review'', ''planned''),
        (''723CS'', ''This change request outlines modifications to the current load balancer configuration in the production environment. The goal is to optimize traffic distribution across the primary application servers (AppSrv-01, AppSrv-02, AppSrv-03) to improve system reliability during high traffic spikes. Current Setup: The load balancer (LB-Prod) is currently set to a round-robin algorithm with a static weight of 1 for each server. Proposed Change: Implement a dynamic load balancing algorithm using least connections for improved resource distribution based on real-time traffic. Configuration Details: Modify the load balancing strategy from round-robin to Least Connections for AppSrv-01, AppSrv-02, and AppSrv-03. Increase the weight of AppSrv-03 (currently at weight 1) to weight 2, as it is the most powerful server. Implement automatic failover configuration in the event of server downtime to reduce service interruptions. Adjust session persistence settings for clients with long transactions to be sticky to the same backend server. Testing will be conducted on a staging environment (LB-Staging) to ensure optimal performance before full deployment. Expected downtime is minimal and will occur during the initial configuration phase.'', ''2025-02-03'', ''Improved system reliability with minimal downtime'', ''within release window'', ''moderate risk'', ''patch'', ''under review'', ''manager override'', ''unplanned''),
        (''6D62EE'', ''This change request aims to reconfigure the existing MySQL database cluster (DBCluster-Prod) to support horizontal scaling by introducing a sharded database model. This will help improve read and write throughput, especially for the rapidly growing user base in the e-commerce application. Current Setup: The current setup consists of a single master node (DBMaster-Prod) and two read replicas (DBReplica1, DBReplica2). Proposed Change: Introduce horizontal sharding by partitioning the user data across four separate database instances: DBShard-01, DBShard-02, DBShard-03, DBShard-04. Implement MySQL ProxySQL to handle routing of read and write queries to the appropriate shard based on the user’s region. Set up automatic sharding management to allow for easy redistribution of data as new shards are added in the future. Deploy a Data Sync Tool for initial data migration from DBMaster-Prod to the new shards. Update application logic to support sharded database queries, ensuring that application components such as user authentication and order management are correctly routed to the right database shards. Testing will be conducted in a dedicated staging environment (DBCluster-Staging) to verify data integrity and application performance. A backup of all critical data will be taken before the migration begins. Expected downtime is approximately 30 minutes for initial shard setup.'', ''2024-02-03'', ''Improved read and write throughput with minimal downtime'', ''after approval'', ''high risk'', ''planned release as part of project koala'', ''approved'', ''uniform agreement'', ''emergency'');
    </sample_insert_statement>
    
    <task>
        Considering the metadata, the sample insert statements, and your role, please create {rows} more lines of insert statements.
    </task>

    <output>
        Absolutely only return the sql and no other wording.
    </output>
    """
    return prompt

# Generate DDL

Using those functions, let's create the DDL to insert values into our table.

In [ ]:
import streamlit as st
from snowflake.cortex import Complete
from snowflake.snowpark.functions import col, call_udf
from snowflake.snowpark.context import get_active_session
import json5 as json

#get a session
session = get_active_session()
#select a model
model = 'llama3.1-405b'

#get first prompt which can be used to create the table AND generate insert statements
initial_data_prompt = generate_data_prompt()

#get the second prompt which will just generate the insert statements
more_data_prompt = generate_more_prompt(number_of_rows)

#set temperature which handle the varilablity in the model inference (we want this fairly high for this use case so we get a wide variety of syntheic data)
temperature = .85

#set the max tokens to 200K so that we do not truncate any responses
max_tokens = 200000

#set the number of rows we want the model to create
number_of_rows = 45

df = my_complete(model, more_data_prompt, temperature, max_tokens)

#show us the data-- we can just copy that and paste it into cells to get the data
#run this as many times as you want to fill the table with as much data as you would like

st.write(df)

# Build Out Our Environment

In [ ]:
CREATE DATABASE IF NOT EXISTS GEN_AI_FSI;
CREATE SCHEMA IF NOT EXISTS GEN_AI_FSI.DTCC_HACKATHON;

USE DATABASE GEN_AI_FSI;
USE SCHEMA GEN_AI_FSI.DTCC_HACKATHON;

CREATE OR TABLE IF NOT EXISTS GEN_AI_FSI.DTCC_HACKATHON.CHANGE_REQUEST_RAW (
       ChangeNumber VARCHAR(100),
       Description VARCHAR(16777216),
       Date DATE,
       Impact VARCHAR(4000),
       Priority VARCHAR(400),
       Risk VARCHAR(400),
       Justification VARCHAR(4000),
       State VARCHAR(400),
       Disposition VARCHAR(400),
       Category VARCHAR(400)
   );

# Insert Data Into CHANGE_REQUEST_RAW

In [ ]:
INSERT INTO GEN_AI_FSI.DTCC_HACKATHON.CHANGE_REQUEST_RAW (ChangeNumber, Description, Date, Impact, Priority, Risk, Justification, State, Disposition, Category) VALUES
   ('AB1672', 'This change request proposes an upgrade to the existing DataSync API from version 3.2 to 3.4 in the Production environment. The update includes several key performance optimizations, enhanced security features, and bug fixes that address issues with data consistency and processing time. The major components of this change include: API Version Update: Migrating from DataSync API v3.2 to v3.4 to support faster data ingestion and processing. Security Enhancements: Implementation of OAuth 2.0-based authentication to replace the legacy basic authentication mechanism, improving overall security for API transactions. Error Handling: Enhanced error codes and more descriptive responses for improved debugging in the event of failure. Database Schema Update: Modifications to the backend MySQL database to accommodate new data types introduced in version 3.4. Testing will be performed in the staging environment (v3.4-Stage) before deployment to ensure backwards compatibility with existing systems. No downtime is expected during the deployment, but a rollback plan has been prepared in case of critical issues.', '2024-11-11', 'Improved performance and security with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'),
   ('723CS', 'This change request outlines modifications to the current load balancer configuration in the production environment. The goal is to optimize traffic distribution across the primary application servers (AppSrv-01, AppSrv-02, AppSrv-03) to improve system reliability during high traffic spikes. Current Setup: The load balancer (LB-Prod) is currently set to a round-robin algorithm with a static weight of 1 for each server. Proposed Change: Implement a dynamic load balancing algorithm using least connections for improved resource distribution based on real-time traffic. Configuration Details: Modify the load balancing strategy from round-robin to Least Connections for AppSrv-01, AppSrv-02, and AppSrv-03. Increase the weight of AppSrv-03 (currently at weight 1) to weight 2, as it is the most powerful server. Implement automatic failover configuration in the event of server downtime to reduce service interruptions. Adjust session persistence settings for clients with long transactions to be sticky to the same backend server. Testing will be conducted on a staging environment (LB-Staging) to ensure optimal performance before full deployment. Expected downtime is minimal and will occur during the initial configuration phase.', '2024-11-11', 'Improved system reliability with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'),
   ('6D62EE', 'This change request aims to reconfigure the existing MySQL database cluster (DBCluster-Prod) to support horizontal scaling by introducing a sharded database model. This will help improve read and write throughput, especially for the rapidly growing user base in the e-commerce application. Current Setup: The current setup consists of a single master node (DBMaster-Prod) and two read replicas (DBReplica1, DBReplica2). Proposed Change: Introduce horizontal sharding by partitioning the user data across four separate database instances: DBShard-01, DBShard-02, DBShard-03, DBShard-04. Implement MySQL ProxySQL to handle routing of read and write queries to the appropriate shard based on the user’s region. Set up automatic sharding management to allow for easy redistribution of data as new shards are added in the future. Deploy a Data Sync Tool for initial data migration from DBMaster-Prod to the new shards. Update application logic to support sharded database queries, ensuring that application components such as user authentication and order management are correctly routed to the right database shards. Testing will be conducted in a dedicated staging environment (DBCluster-Staging) to verify data integrity and application performance. A backup of all critical data will be taken before the migration begins. Expected downtime is approximately 30 minutes for initial shard setup.', '2024-11-11', 'Improved read and write throughput with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'),
 ('AB1673', 'This change request proposes a server migration from the legacy physical servers (Srv-Old-01, Srv-Old-02) to a new virtualized environment in VMware vSphere. The migration will improve resource allocation, reduce hardware costs, and provide better scalability. The new setup will include 4 VMs (VM-01 to VM-04) configured for redundancy and load balancing. Initial migration testing will be done on the staging environment (VMware-Staging), with a 2-hour maintenance window required for the final migration.', '2024-11-11', 'Improved scalability and cost savings', 'within maintenance window', 'moderate risk', 'cost optimization', 'approved', 'no peer review', 'planned'),
   ('AB1674', 'This change request outlines the integration of the new Customer Relationship Management (CRM) tool into the existing infrastructure. The CRM tool, Salesforce Cloud, will replace the legacy in-house system. The integration will involve data migration, API connections to internal systems, and user training. The migration process will begin in the staging environment, followed by a phased rollout to the production environment.', '2024-11-11', 'Enhanced customer interaction and reporting capabilities', 'within release window', 'low risk', 'system replacement', 'new', 'peer-reviewed', 'planned'),
   ('AB1675', 'This change request involves patching the security vulnerabilities in the web application firewall (WAF) configuration. The update will address critical vulnerabilities identified in the recent security audit. Changes will include tightening rules for SQL injection and cross-site scripting (XSS), and enhancing logging for monitoring suspicious activity. Testing will be done in the staging environment before pushing the update live in production.', '2024-11-11', 'Enhanced security of web applications', 'immediately', 'high risk', 'security patch', 'under review', 'manager override', 'unplanned'),
   ('AB1676', 'This change request proposes the upgrade of the Apache web server from version 2.4.46 to 2.4.54. The upgrade includes performance improvements, security patches, and support for HTTP/3. Additionally, several deprecated modules will be removed, and new SSL/TLS certificates will be installed. No downtime is expected, but a rollback plan is in place for potential issues.', '2024-11-11', 'Improved performance and security', 'within release window', 'low risk', 'system upgrade', 'new', 'no peer review', 'planned'),
   ('AB1677', 'This change request involves expanding the cloud storage capacity on AWS S3 by 5 TB to accommodate growing user data from the mobile app. The expansion will be managed using an automated script to ensure consistent provisioning, and backups will be taken before initiating the process. The expansion is expected to be seamless with minimal user impact.', '2024-11-11', 'Increased storage capacity with no downtime', 'within maintenance window', 'low risk', 'storage expansion', 'approved', 'peer-reviewed', 'unplanned'),
   ('AB1678', 'This change request suggests a reconfiguration of the DNS settings for the company’s public-facing websites. The objective is to improve redundancy and performance by integrating Route 53 with health checks and failover routing. The change will be tested in a sandbox environment before being pushed live to avoid downtime during the migration.', '2024-11-11', 'Improved availability and performance', 'within release window', 'moderate risk', 'performance optimization', 'under review', 'no peer review', 'planned'),
   ('AB1679', 'This change request proposes a new backup solution using Veeam Backup & Replication. The current backup system is inefficient and requires manual intervention. The new solution will automate backup jobs, include replication to an offsite location, and implement encryption for secure data storage. The initial setup and testing will occur in the staging environment before going live.', '2024-11-11', 'Automated backups with enhanced security', 'within maintenance window', 'low risk', 'new system installation', 'approved', 'manager override', 'planned'),
   ('AB1680', 'This change request involves the migration of the internal email system from Exchange Server 2016 to Microsoft 365. This migration will enable enhanced collaboration features, improve user experience, and reduce on-premises maintenance. The migration will be carried out in stages, starting with a pilot group, and full deployment will occur over the course of 2 weeks. A fallback plan is in place in case of service disruption.', '2024-11-11', 'Improved email functionality and collaboration tools', 'after approval', 'moderate risk', 'service migration', 'new', 'peer-reviewed', 'unplanned'),
   ('AB1681', 'This change request aims to upgrade the enterprise network infrastructure by replacing the aging core switches with Cisco Catalyst 9300 models. The upgrade will enhance network speed, support higher throughput, and improve security features such as 802.1X authentication. The work will be scheduled for a weekend to minimize downtime and will involve several phases of testing.', '2024-11-11', 'Improved network performance and security', 'within release window', 'high risk', 'infrastructure upgrade', 'approved', 'uniform agreement', 'planned'),
   ('AB1682', 'This change request proposes to integrate the payment gateway API (Stripe) into the company’s e-commerce platform to replace the legacy payment processor. The integration will include testing for PCI DSS compliance, customer notification, and system load testing to ensure performance during peak traffic periods. The rollout will happen incrementally, with extensive monitoring during the first week.', '2024-11-11', 'Enhanced payment processing capabilities', 'immediately', 'moderate risk', 'service upgrade', 'new', 'no peer review', 'unplanned'),
('AB1673', 'This change request proposes the migration of internal file storage from an on-premises SAN (Storage Area Network) to Amazon S3 for better scalability and cost optimization. The process will involve the migration of over 10 TB of data using AWS DataSync, with a staged migration plan that includes data validation, checksum comparisons, and testing to ensure integrity. The new solution will leverage S3 lifecycle policies for automated archival and deletion, and a CloudFront CDN will be configured for faster content delivery. The migration will happen in batches, with a 48-hour downtime window per batch, during which all file-related services will be unavailable.', '2024-11-11', 'Enhanced scalability and reduced operational cost', 'within release window', 'moderate risk', 'storage migration', 'approved', 'manager override', 'unplanned'),
   ('AB1674', 'This change request involves the reconfiguration of the Kubernetes cluster running in Google Kubernetes Engine (GKE). The changes will include increasing the number of nodes in the cluster from 5 to 15 to meet the increasing load of microservices. The configuration will also include moving to a multi-zone setup to enhance fault tolerance. Helm charts will be updated for automated deployment of containerized applications. Additionally, NodePools will be reconfigured to include GPU-enabled nodes for AI processing workloads. The new setup will leverage GKE Autopilot mode to ensure optimal resource utilization and autoscaling.', '2024-11-11', 'Improved application performance and scalability', 'immediately', 'high risk', 'cluster scaling', 'under review', 'peer-reviewed', 'planned'),
   ('AB1675', 'This change request proposes a complete overhaul of the internal load balancing mechanism for the web application deployed across AWS Elastic Load Balancers (ELB). The current setup uses a classic load balancer with static routing, which has shown limitations during high-traffic events. The new architecture will involve configuring an Application Load Balancer (ALB) with path-based routing and host-based routing to improve traffic management. AWS WAF (Web Application Firewall) rules will be enhanced to mitigate against OWASP Top 10 vulnerabilities. Additionally, integration with AWS Lambda will enable real-time anomaly detection, with traffic rerouting to spare instances during detected spikes.', '2024-11-11', 'Improved traffic management and security', 'within release window', 'high risk', 'traffic optimization', 'approved', 'uniform agreement', 'planned'),
   ('AB1676', 'This change request details the upgrade of the enterprise Active Directory (AD) infrastructure from Windows Server 2016 to 2022 to support modern authentication protocols like FIDO2 and passwordless login. The update will include the migration of all domain controllers to new virtual machines hosted on VMware vSphere 7.0, with the introduction of a read-only domain controller (RODC) at remote offices for improved security. Group Policy Objects (GPOs) will be reviewed and refactored to comply with the latest security hardening guidelines. Additionally, the AD Federation Services (ADFS) will be upgraded to support integration with Azure AD for hybrid cloud management.', '2024-11-11', 'Enhanced security and modernized authentication', 'within release window', 'moderate risk', 'security enhancement', 'approved', 'peer-reviewed', 'planned'),
   ('AB1677', 'This change request involves the deployment of a multi-region Redis cluster to improve data caching and reduce latency for geographically distributed users. The Redis cluster will be deployed across AWS regions using the ElastiCache service, with automatic failover and cross-region replication to ensure high availability. Each region will host a master Redis node, with slave nodes configured for read scaling. The Redis setup will also be integrated with AWS CloudWatch for real-time performance monitoring and alerting. The implementation will occur during the planned maintenance window, with minimal downtime expected due to pre-warming of the cache nodes.', '2024-11-11', 'Improved data caching and reduced latency', 'within release window', 'moderate risk', 'performance optimization', 'approved', 'no peer review', 'unplanned'),
   ('AB1678', 'This change request proposes the integration of a new AI-based fraud detection module into the transactional systems of the company’s banking platform. The AI model, built with TensorFlow and trained on historical transaction data, will run as a microservice within Kubernetes. The integration will involve the configuration of RESTful API endpoints using Kong API Gateway, which will interface with the transactional database in PostgreSQL. The system will use machine learning techniques such as anomaly detection and supervised learning to flag high-risk transactions in real-time. Full integration will be done in the staging environment for two weeks before going live.', '2024-11-11', 'Enhanced fraud detection capabilities', 'immediately', 'high risk', 'service integration', 'new', 'peer-reviewed', 'planned'),
   ('AB1679', 'This change request involves the transition from traditional on-premises VMware ESXi hosts to a fully managed Kubernetes container orchestration platform in Microsoft Azure Kubernetes Service (AKS). The shift will include replatforming legacy Java-based applications, which are currently running on VMs, to microservices within Kubernetes clusters. ACI (Azure Container Instances) will be used to handle burst workloads, while Azure Blob Storage will be used to manage static data. The deployment will utilize Azure DevOps pipelines for CI/CD automation and will implement Azure Monitor for application performance tracking. The final migration will happen after load testing and validation in the Azure staging environment.', '2024-11-11', 'Improved scalability and cloud-native deployment', 'after approval', 'high risk', 'cloud migration', 'approved', 'manager override', 'unplanned'),
   ('AB1680', 'This change request is focused on the implementation of a centralized logging system using the ELK stack (Elasticsearch, Logstash, Kibana) to collect and analyze application logs across all microservices in the production environment. The ELK cluster will be set up in a dedicated VPC to segregate the traffic from other application components. Logstash will be configured to parse logs from various services, including web servers, application servers, and databases, with Elasticsearch providing the backend for querying logs. Kibana dashboards will be set up for real-time visualization of log data, with automated alerts for anomaly detection. The setup will be tested first in the staging environment before production deployment.', '2024-11-11', 'Centralized logging and real-time monitoring', 'within maintenance window', 'low risk', 'monitoring system installation', 'approved', 'peer-reviewed', 'planned'),
   ('AB1681', 'This change request proposes the configuration of a distributed tracing system using OpenTelemetry and Jaeger to track and optimize the performance of microservices running on AWS Fargate. The Jaeger collector will be deployed in a highly available setup using EC2 instances across multiple availability zones. Application code will be instrumented with OpenTelemetry SDKs for distributed tracing, and traces will be stored in Amazon RDS for historical analysis. The system will also use AWS X-Ray for integration with AWS-native services. Real-time performance metrics will be visible in Grafana dashboards, and alerts will be configured for high latency or failures.', '2024-11-11', 'Improved application performance and observability', 'within release window', 'moderate risk', 'performance optimization', 'under review', 'uniform agreement', 'planned'),
   ('AB1682', 'This change request involves the update of the firewall rules and access control lists (ACLs) in the internal network to tighten security and comply with new regulatory requirements. The firewall will be reconfigured to allow only specific IP ranges from known partners and block all unsolicited inbound traffic. Additionally, VLANs will be configured to segment traffic between different departments, and redundant firewall appliances will be deployed for failover. The changes will be tested in a lab environment before implementation, with logs being forwarded to a Splunk server for real-time analysis.', '2024-11-11', 'Enhanced network security and regulatory compliance', 'immediately', 'high risk', 'security hardening', 'new', 'no peer review', 'unplanned'),
   ('AB1683', 'This change request focuses on upgrading the company’s legacy virtual desktop infrastructure (VDI) from Citrix XenDesktop 7.15 to Citrix Virtual Apps and Desktops 2203. The upgrade will include a complete re-architecting of the Citrix Delivery Controller, Citrix StoreFront, and Citrix Workspace, with the migration of all user profiles and settings to the new environment. Citrix ADC (formerly NetScaler) will also be upgraded to improve session management and load balancing. The new environment will be configured for high availability using Citrix Cloud and integrated with Microsoft Azure AD for single sign-on capabilities. Full user acceptance testing will be conducted in a parallel environment before the final migration.', '2024-11-11', 'Improved virtual desktop experience and scalability', 'within maintenance window', 'moderate risk', 'VDI upgrade', 'approved', 'peer-reviewed', 'planned');
 

In [ ]:
INSERT INTO GEN_AI_FSI.DTCC_HACKATHON.CHANGE_REQUEST_RAW (ChangeNumber, Description, Date, Impact, Priority, Risk, Justification, State, Disposition, Category) VALUES ('FB457A', 'Implement a new machine learning model for fraud detection in the payment system. The model will use advanced algorithms and data preprocessing techniques to reduce false positives and improve overall fraud detection accuracy. The model will be deployed across all payment processing environments (Staging, Production) to ensure consistent and timely fraud detection. The model will be continuously monitored and updated with new data to improve its performance over time.', '2025-03-15', 'Improved fraud detection accuracy with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('9C89DF', 'Upgrade the existing network infrastructure to support higher bandwidth and lower latency for critical applications. The upgrade will include the installation of new switches, routers, and fiber optic cables. The upgrade will be performed during off-peak hours to minimize potential disruption to users. Testing will be conducted in a dedicated staging environment to ensure optimal performance before full deployment. The expected downtime during the upgrade is approximately 4 hours.', '2025-04-01', 'Improved network performance with minimal disruption', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('3D27EB', 'Introduce a new data pipeline for real-time analysis of social media data. The pipeline will use Apache Kafka for data ingestion, Apache Spark for data processing, and Apache Hadoop for data storage. The pipeline will be designed to handle high volumes of data in real-time, providing insights into sentiment analysis, trending topics, and user behavior. The pipeline will be deployed across Staging and Production environments to ensure consistent and reliable data processing. Testing will be conducted in a dedicated staging environment to verify data accuracy and pipeline performance.', '2025-05-01', 'Real-time social media data analysis with minimal disruption', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('B6E8AC', 'Migrate the existing Oracle database to a more cost-effective and scalable solution using Amazon Aurora. The migration will be performed using Oracles Data Pump utility to minimize data loss and downtime. Testing will be conducted in a dedicated staging environment to ensure data consistency and performance. The expected downtime during the migration is approximately 2 hours.', '2025-06-01', 'Improved database scalability with cost savings', 'within release window', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('F4C8B9', 'Upgrade the existing web server infrastructure to support Http/2 protocol for improved performance and reduced latency. The upgrade will include the installation of the latest version of Apache HTTP Server and enabling Http/2 support. The upgrade will be performed across all web server environments (Staging, Production) to ensure consistent and reliable performance. Testing will be conducted in a dedicated staging environment to verify performance improvements. The expected downtime during the upgrade is approximately 1 hour.', '2025-07-01', 'Improved web server performance and reduced latency', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('1BEC7A', 'Introduce a new incident response system to quickly identify and mitigate security incidents. The system will use machine learning techniques to identify potential security threats and provide real-time alerts to security analysts. The system will be deployed across all security monitoring environments (Staging, Production) to ensure consistent and reliable incident response. Testing will be conducted in a dedicated staging environment to verify the systems accuracy and performance. The expected downtime during the deployment is minimal.', '2025-08-01', 'Improved security incident response with minimal disruption', 'within release window', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('5D8F13', 'Upgrade the existing virtualization infrastructure to support the latest version of VMware vSphere. The upgrade will include the installation of new hypervisor nodes and updated software components. The upgrade will be performed during off-peak hours to minimize potential disruption to virtualized applications. Testing will be conducted in a dedicated staging environment to ensure optimal performance before full deployment. The expected downtime during the upgrade is approximately 6 hours.', '2025-09-01', 'Improved virtualization performance and scalability', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('8F3D67', 'Introduce a new data visualization dashboard for business intelligence reporting. The dashboard will use Tableau for data visualization and will provide real-time insights into key business metrics. The dashboard will be deployed across all reporting environments (Staging, Production) to ensure consistent and reliable data visualization. Testing will be conducted in a dedicated staging environment to verify data accuracy and dashboard performance. The expected downtime during the deployment is minimal.', '2025-10-01', 'Improved business intelligence reporting with minimal disruption', 'after approval', 'low risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('E2C97B', 'Upgrade the existing messaging system to support the latest version of Apache Kafka. The upgrade will include the installation of new brokers and updated software components. The upgrade will be performed across all messaging environments (Staging, Production) to ensure consistent and reliable message processing. Testing will be conducted in a dedicated staging environment to verify performance improvements. The expected downtime during the upgrade is approximately 2 hours.', '2025-11-01', 'Improved message processing with minimal disruption', 'within release window', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('A6B9E4', 'Introduce a new container orchestration system using Kubernetes. The system will provide automated deployment and scaling of containerized applications, reducing manual intervention and improving overall application availability. The system will be deployed across all deployment environments (Staging, Production) to ensure consistent and reliable application deployment. Testing will be conducted in a dedicated staging environment to verify system accuracy and performance. The expected downtime during the deployment is minimal.', '2025-12-01', 'Improved container orchestration with minimal disruption', 'after approval', 'low risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency');

In [ ]:
INSERT INTO GEN_AI_FSI.DTCC_HACKATHON.CHANGE_REQUEST_RAW (ChangeNumber, Description, Date, Impact, Priority, Risk, Justification, State, Disposition, Category) VALUES ('A12345', 'Upgrading database server from MySQL 8.0 to MySQL 8.1 in Production environment. The update includes bug fixes and improved security features for better data protection. The major components of this change include: Server Version Update: Updating MySQL server from version 8.0 to 8.1 to address known bugs and vulnerabilities. Security Enhancements: Implementing improved encryption protocols to strengthen data security. Database Schema Update: Migrating data to a new schema that optimizes read and write operations. Testing will be performed in the staging environment (MySQL-8.1-Staging) before deployment to ensure compatibility with existing systems. No downtime is expected during the deployment.', '2024-02-02', 'Improved database performance and security with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('B23456', 'Modifying network infrastructure for better data flow in Data Center A. The changes include upgrading network switches and enhancing network security to mitigate cyber threats. The major components of this change include: Network Switch Upgrade: Replacing aging switches with new high-performance switches that support faster data transfer rates. Network Security Enhancement: Implementing advanced threat detection and prevention measures to safeguard network infrastructure. Testing will be conducted in a staging environment (DataCenter-A-Staging) to verify network performance and visibility. Downtime is expected to be around 2 hours during the upgrade process.', '2024-02-01', 'Improved network performance and security with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('C34567', 'Implementing a new backup solution for critical data in Production environment. The solution optimizes data replication and recovery processes for improved business continuity. The major components of this change include: Backup Solution Integration: Integrating a new backup solution (Veeam Backup & Replication) to replace the current solution (Commvault). Data Replication Optimization: Configuring the new solution to replicate critical data to a secondary location for improved data recovery. Testing will be performed in the staging environment (Backup-Staging) to ensure smooth operation and data integrity. No downtime is expected during the deployment.', '2024-01-31', 'Improved data replication and recovery process with minimal downtime', 'after approval', 'high risk', 'planned release as part of project ocelot', 'approved', 'uniform agreement', 'emergency'), ('D45678', 'Upgrading storage infrastructure for the cloud-based application. The change includes migrating from on-premises storage to a cloud-based storage solution for enhanced scalability and reliability. The major components of this change include: Storage Infrastructure Migration: Migrating data from on-premises storage to a cloud-based storage solution (AWS S3). Data Sync Tool Setup: Configuring a data sync tool for real-time data synchronization between the on-premises and cloud-based storage. Testing will be performed in the staging environment (CloudStorage-Staging) to ensure data integrity and application performance. Expected downtime is approximately 1 hour during the migration process.', '2024-01-30', 'Improved scalability and reliability with minimal downtime', 'within release window', 'moderate risk', 'planned release', 'approved', 'no peer review', 'planned'), ('E56789', 'Implementing a new disaster recovery solution for the web application. The solution includes replicating data to a secondary data center for improved data recovery in case of a catastrophic event. The major components of this change include: Disaster Recovery Infrastructure Setup: Setting up a secondary data center (DR Site) with all necessary hardware and software components to replicate data in real-time from the primary data center. Data Replication Optimization: Configuring the replication tool for real-time data synchronization between the primary and secondary data centers. Testing will be conducted in a staging environment (DR-Staging) to verify data replication and recovery process. No downtime is expected during the deployment.', '2024-01-29', 'Improved data recovery process with minimal downtime', 'after approval', 'high risk', 'planned release as part of project lynx', 'approved', 'uniform agreement', 'emergency'), ('F67890', 'Integrating a new API (Weather API) into the existing e-commerce application. The API provides real-time weather data for personalized user experience. The major components of this change include: API Integration: Integrating the Weather API into the existing e-commerce application. Data Sync Tool Setup: Configuring a data sync tool for real-time weather data synchronization between the API and the application. Testing will be performed in the staging environment (WeatherAPI-Staging) to ensure smooth operation and user experience. No downtime is expected during the deployment.', '2024-01-28', 'Improved user experience with real-time weather data', 'within release window', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('G78901', 'Upgrading the content management system (CMS) for the corporate website. The upgrade includes new features and improved performance for better user experience. The major components of this change include: CMS Version Update: Upgrading the CMS from version 5.3 to 5.5 to support new features and performance improvements. User Interface Enhancements: Implementing new UI components and layouts for improved user experience. Performance Optimization: Optimizing database queries and server-side logic for better response time. Testing will be performed in the staging environment (CMS-Staging) to ensure smooth operation and user experience. No downtime is expected during the deployment.', '2024-01-27', 'Improved CMS performance and user experience with minimal downtime', 'within release window', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('H89012', 'Implementing a new payment gateway for the e-commerce application. The gateway supports multiple payment options and provides enhanced fraud detection features. The major components of this change include: Payment Gateway Integration: Integrating a new payment gateway (Stripe) into the e-commerce application. Fraud Detection Enhancement: Implementing advanced fraud detection measures to safeguard user accounts and transactions. Testing will be performed in the staging environment (PaymentGateway-Staging) to ensure smooth operation and transaction security. No downtime is expected during the deployment.', '2024-01-26', 'Improved payment options and transaction security with minimal downtime', 'within release window', 'moderate risk', 'planned release', 'approved', 'no peer review', 'planned'), ('I90123', 'Updating the logging infrastructure for all web applications. The change includes implementing centralized logging and log analysis for better troubleshooting and performance monitoring. The major components of this change include: Logging Infrastructure Migration: Migrating all web application logs to a centralized ELK (Elasticsearch, Logstash, Kibana) stack for better log analysis. Log Filter Configuration: Configuring log filters to filter out unnecessary log messages for better performance. Testing will be performed in the staging environment (Logging-Staging) to ensure smooth operation and performance monitoring. No downtime is expected during the deployment.', '2024-01-25', 'Improved troubleshooting and performance monitoring with minimal downtime', 'within release window', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('J01234', 'Upgrading the network security infrastructure for better threat detection and prevention. The change includes implementing new firewalls and intrusion detection systems for improved network security. The major components of this change include: Firewall Upgrade: Replacing aging firewalls with new ones that support advanced threat detection and prevention capabilities. Intrusion Detection System Implementation: Implementing a new intrusion detection system (IDS) to monitor and analyze network traffic for potential threats. Testing will be performed in the staging environment (NetworkSecurity-Staging) to ensure smooth operation and threat detection. Expected downtime is approximately 4 hours during the upgrade process.', '2024-01-24', 'Improved network security with minimal downtime', 'within release window', 'moderate risk', 'planned release', 'approved', 'no peer review', 'planned'), ('K12345', 'Implementing a new identity and access management (IAM) solution for better user authentication and authorization. The solution includes integrating a new IAM service (Okta) into all web applications. The major components of this change include: IAM Integration: Integrating a new IAM service (Okta) for user authentication and authorization. API Configuration: Configuring the Okta API for seamless integration with all web applications. Testing will be performed in the staging environment (IAM-Staging) to ensure smooth operation and security. No downtime is expected during the deployment.', '2024-01-23', 'Improved user authentication and authorization with minimal downtime', 'within release window', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('L23456', 'Implementing a new alerting solution for better system monitoring and issue notification. The solution includes integrating a new alerting service (Pagerduty) into all web applications. The major components of this change include: Alerting Service Integration: Integrating a new alerting service (Pagerduty) for system monitoring and issue notification. Rules Configuration: Configuring alerting rules for seamless integration with all web applications. Testing will be performed in the staging environment (Alerting-Staging) to ensure smooth operation and issue notification. No downtime is expected during the deployment.', '2024-01-22', 'Improved system monitoring and issue notification with minimal downtime', 'within release window', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('M34567', 'Updating the network infrastructure for better network connectivity and performance. The change includes upgrading network routers and switches for improved network performance. The major components of this change include: Network Router Upgrade: Replacing aging routers with new ones that support faster data transfer rates. Network Switch Upgrade: Replacing aging switches with new ones that support faster data transfer rates. Testing will be performed in the staging environment (NetworkInfrastructure-Staging) to ensure smooth operation and network performance. No downtime is expected during the deployment.', '2024-01-21', 'Improved network connectivity and performance with minimal downtime', 'within release window', 'moderate risk', 'planned release', 'approved', 'no peer review', 'planned'), ('N45678', 'Implementing a new database monitoring solution for better database performance and troubleshooting. The solution includes integrating a new database monitoring tool (Datadog) into all databases. The major components of this change include: Database Monitoring Tool Integration: Integrating a new database monitoring tool (Datadog) for better database performance and troubleshooting. Metrics Collection Configuration: Configuring metrics collection for seamless integration with all databases. Testing will be performed in the staging environment (DatabaseMonitoring-Staging) to ensure smooth operation and database performance. No downtime is expected during the deployment.', '2024-01-20', 'Improved database performance and troubleshooting with minimal downtime', 'within release window', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('O56789', 'Upgrading the security of all web applications by implementing new encryption protocols. The change includes implementing end-to-end encryption for all user data. The major components of this change include: Encryption Protocol Update: Implementing end-to-end encryption for all user data. Web Application Code Update: Updating all web application code to support the new encryption protocols. Testing will be performed in the staging environment (Security-Staging) to ensure smooth operation and data security. No downtime is expected during the deployment.', '2024-01-19', 'Improved data security with minimal downtime', 'within release window', 'high risk', 'planned release as part of project tundra', 'approved', 'uniform agreement', 'emergency'), ('P67890', 'Implementing a new file storage solution for better file management and file retrieval. The solution includes integrating a new file storage service (Amazon S3) into all web applications. The major components of this change include: File Storage Service Integration: Integrating a new file storage service (Amazon S3) for better file management and file retrieval. File Retrieval Optimization: Configuring file retrieval settings for faster file retrieval. Testing will be performed in the staging environment (FileStorage-Staging) to ensure smooth operation and file retrieval. No downtime is expected during the deployment.', '2024-01-18', 'Improved file management and file retrieval with minimal downtime', 'within release window', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('Q78901', 'Upgrading the web server infrastructure for better web application performance. The change includes upgrading the web server software to the latest version. The major components of this change include: Web Server Software Upgrade: Upgrading the web server software to the latest version for better web application performance. Web Application Code Update: Updating all web application code to support the latest web server software. Testing will be performed in the staging environment (WebServer-Staging) to ensure smooth operation and web application performance. No downtime is expected during the deployment.', '2024-01-17', 'Improved web application performance with minimal downtime', 'within release window', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('R89012', 'Implementing a new microservices architecture for better scalability and maintainability. The change includes implementing a new service discovery tool (Consul) for better service coordination and communication. The major components of this change include: Service Discovery Tool Integration: Integrating a new service discovery tool (Consul) for better service coordination and communication. Microservices Architecture Implementation: Implementing a new microservices architecture for better scalability and maintainability. Testing will be performed in the staging environment (Microservices-Staging) to ensure smooth operation and scalability. No downtime is expected during the deployment.', '2024-01-16', 'Improved scalability and maintainability with minimal downtime', 'within release window', 'moderate risk', 'planned release', 'approved', 'no peer review', 'planned'), ('S90123', 'Updating the network security infrastructure for better threat detection and prevention. The change includes implementing new firewalls and intrusion detection systems for improved network security. The major components of this change include: Firewall Upgrade: Replacing aging firewalls with new ones that support advanced threat detection and prevention capabilities. Intrusion Detection System Implementation: Implementing a new intrusion detection system (IDS) to monitor and analyze network traffic for potential threats. Testing will be performed in the staging environment (NetworkSecurity-Staging) to ensure smooth operation and threat detection. Expected downtime is approximately 4 hours during the upgrade process.', '2024-01-15', 'Improved network security with minimal downtime', 'within release window', 'moderate risk', 'planned release', 'approved', 'no peer review', 'planned'), ('T01234', 'Implementing a new identity and access management (IAM) solution for better user authentication and authorization. The solution includes integrating a new IAM service (Okta) into all web applications. The major components of this change include: IAM Integration: Integrating a new IAM service (Okta) for user authentication and authorization. API Configuration: Configuring the Okta API for seamless integration with all web applications. Testing will be performed in the staging environment (IAM-Staging) to ensure smooth operation and security. No downtime is expected during the deployment.', '2024-01-14', 'Improved user authentication and authorization with minimal downtime', 'within release window', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('U12345', 'Implementing a new alerting solution for better system monitoring and issue notification. The solution includes integrating a new alerting service (Pagerduty) into all web applications. The major components of this change include: Alerting Service Integration: Integrating a new alerting service (Pagerduty) for system monitoring and issue notification. Rules Configuration: Configuring alerting rules for seamless integration with all web applications. Testing will be performed in the staging environment (Alerting-Staging) to ensure smooth operation and issue notification. No downtime is expected during the deployment.', '2024-01-13', 'Improved system monitoring and issue notification with minimal downtime', 'within release window', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('V23456', 'Updating the logging infrastructure for all web applications. The change includes implementing centralized logging and log analysis for better troubleshooting and performance monitoring. The major components of this change include: Logging Infrastructure Migration: Migrating all web application logs to a centralized ELK (Elasticsearch, Logstash, Kibana) stack for better log analysis. Log Filter Configuration: Configuring log filters to filter out unnecessary log messages for better performance. Testing will be performed in the staging environment (Logging-Staging) to ensure smooth operation and performance monitoring. No downtime is expected during the deployment.', '2024-01-12', 'Improved troubleshooting and performance monitoring with minimal downtime', 'within release window', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('W34567', 'Upgrading the content management system (CMS) for the corporate website. The upgrade includes new features and improved performance for better user experience. The major components of this change include: CMS Version Update: Upgrading the CMS from version 5.3 to 5.5 to support new features and performance improvements. User Interface Enhancements: Implementing new UI components and layouts for improved user experience. Performance Optimization: Optimizing database queries and server-side logic for better response time. Testing will be performed in the staging environment (CMS-Staging) to ensure smooth operation and user experience. No downtime is expected during the deployment.', '2024-01-11', 'Improved CMS performance and user experience with minimal downtime', 'within release window', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('X45678', 'Implementing a new payment gateway for the e-commerce application. The gateway supports multiple payment options and provides enhanced fraud detection features. The major components of this change include: Payment Gateway Integration: Integrating a new payment gateway (Stripe) into the e-commerce application. Fraud Detection Enhancement: Implementing advanced fraud detection measures to safeguard user accounts and transactions. Testing will be performed in the staging environment (PaymentGateway-Staging) to ensure smooth operation and transaction security. No downtime is expected during the deployment.', '2024-01-10', 'Improved payment options and transaction security with minimal downtime', 'within release window', 'moderate risk', 'planned release', 'approved', 'no peer review', 'planned'), ('Y56789', 'Integrating a new API (Weather API) into the existing e-commerce application. The API provides real-time weather data for personalized user experience. The major components of this change include: API Integration: Integrating the Weather API into the existing e-commerce application. Data Sync Tool Setup: Configuring a data sync tool for real-time weather data synchronization between the API and the application. Testing will be performed in the staging environment (WeatherAPI-Staging) to ensure smooth operation and user experience. No downtime is expected during the deployment.', '2024-01-09', 'Improved user experience with real-time weather data', 'within release window', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('Z67890', 'Upgrading the storage infrastructure for the cloud-based application. The change includes migrating from on-premises storage to a cloud-based storage solution for enhanced scalability and reliability. The major components of this change include: Storage Infrastructure Migration: Migrating data from on-premises storage to a cloud-based storage solution (AWS S3). Data Sync Tool Setup: Configuring a data sync tool for real-time data synchronization between the on-premises and cloud-based storage. Testing will be performed in the staging environment (CloudStorage-Staging) to ensure data integrity and application performance. Expected downtime is approximately 1 hour during the migration process.', '2024-01-08', 'Improved scalability and reliability with minimal downtime', 'within release window', 'moderate risk', 'planned release', 'approved', 'no peer review', 'planned');

In [ ]:
INSERT INTO GEN_AI_FSI.DTCC_HACKATHON.CHANGE_REQUEST_RAW (ChangeNumber, Description, Date, Impact, Priority, Risk, Justification, State, Disposition, Category) VALUES ('8H123A', 'Upgrade Apache Kafka from version 2.8 to 3.0 in Production environment to improve message queue performance and resolve issues with data loss.', '2025-02-05', 'Improved message queue performance with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('B78BN', 'Modify existing CI/CD pipeline to integrate automated security scanning and compliance checks to improve overall security posture.', '2025-02-08', 'Improved security posture with minimal impact on deployment timelines', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('8N3AA', 'Implement data encryption at rest for all MySQL databases in Production environment to meet new regulatory requirements.', '2025-02-10', 'Improved data security with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('B3R67', 'Reconfigure network firewall rules to restrict incoming traffic to only necessary ports and protocols to improve security posture.', '2025-02-12', 'Improved security posture with minimal impact on system functionality', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('9T7WE', 'Upgrade Jenkins from version 2.3 to 2.5 in Production environment to improve automation capabilities and resolve issues with job scheduling.', '2025-02-15', 'Improved automation capabilities with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('B9U21', 'Implement automated monitoring and alerting for all Production environment systems to improve incident response times.', '2025-02-17', 'Improved incident response times with minimal impact on system functionality', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('7T9AS', 'Reconfigure load balancer to distribute traffic across multiple availability zones to improve system reliability.', '2025-02-19', 'Improved system reliability with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('B1V45', 'Modify existing Docker container configurations to improve resource utilization and reduce costs.', '2025-02-22', 'Improved resource utilization with minimal impact on system functionality', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('8M3DS', 'Implement automated backup and disaster recovery for all Production environment systems to improve business continuity.', '2025-02-24', 'Improved business continuity with minimal impact on system functionality', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('D8EWS', 'Upgrade Python from version 3.9 to 3.10 in Production environment to improve application performance and resolve issues with library compatibility.', '2025-02-26', 'Improved application performance with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('G9UXS', 'Modify existing Kubernetes cluster configurations to improve scalability and reduce costs.', '2025-03-01', 'Improved scalability with minimal impact on system functionality', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('Y6HO5', 'Implement automated compliance scanning for all Production environment systems to improve regulatory compliance.', '2025-03-03', 'Improved regulatory compliance with minimal impact on system functionality', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('8B7V3', 'Reconfigure network routing to improve traffic flow and reduce latency.', '2025-03-05', 'Improved network performance with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('Z1G78', 'Upgrade Ruby from version 2.7 to 3.0 in Production environment to improve application performance and resolve issues with library compatibility.', '2025-03-08', 'Improved application performance with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('H5T12', 'Implement automated security auditing for all Production environment systems to improve security posture.', '2025-03-10', 'Improved security posture with minimal impact on system functionality', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('E6TE2', 'Reconfigure existing storage configurations to improve disk space utilization and reduce costs.', '2025-03-12', 'Improved disk space utilization with minimal impact on system functionality', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('K2V56', 'Modify existing Azure cloud services configurations to improve scalability and reduce costs.', '2025-03-15', 'Improved scalability with minimal impact on system functionality', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('T4AZ7', 'Implement automated incident response for all Production environment systems to improve incident response times.', '2025-03-17', 'Improved incident response times with minimal impact on system functionality', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('F1R23', 'Reconfigure network DNS settings to improve domain name resolution and reduce latency.', '2025-03-19', 'Improved network performance with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('R4EW9', 'Upgrade Node.js from version 14 to 16 in Production environment to improve application performance and resolve issues with library compatibility.', '2025-03-22', 'Improved application performance with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('D5TS3', 'Implement automated problem management for all Production environment systems to improve problem resolution times.', '2025-03-24', 'Improved problem resolution times with minimal impact on system functionality', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('L6BU4', 'Reconfigure existing monitoring configurations to improve system visibility and reduce costs.', '2025-03-26', 'Improved system visibility with minimal impact on system functionality', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('V8RO1', 'Modify existing Google Cloud Platform services configurations to improve scalability and reduce costs.', '2025-03-29', 'Improved scalability with minimal impact on system functionality', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('X2AE5', 'Implement automated release management for all Production environment systems to improve deployment efficiency.', '2025-03-31', 'Improved deployment efficiency with minimal impact on system functionality', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('E3WQ7', 'Reconfigure network VPN settings to improve remote access security and reduce latency.', '2025-04-02', 'Improved network security with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('S1KU9', 'Upgrade Go from version 1.16 to 1.17 in Production environment to improve application performance and resolve issues with library compatibility.', '2025-04-05', 'Improved application performance with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned');

In [ ]:
INSERT INTO GEN_AI_FSI.DTCC_HACKATHON.CHANGE_REQUEST_RAW (ChangeNumber, Description, Date, Impact, Priority, Risk, Justification, State, Disposition, Category) VALUES ('GH6789', 'Upgrade the Jenkins CI/CD pipeline to support parallel builds and improve overall build efficiency.', '2025-03-01', 'Improved build performance with no downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('TY4321', 'Implement automated monitoring and alerting for database query performance to improve issue detection and resolution.', '2025-02-15', 'Improved database performance with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('MN6543', 'Migrate the user authentication system to OAuth 2.0 to improve security and support for third-party integrations.', '2024-03-01', 'Improved security with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('VC3219', 'Replace the existing load balancer with a more efficient and scalable solution to improve system reliability.', '2025-01-15', 'Improved system reliability with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('ZX1234', 'Introduce a caching layer to improve application performance and reduce database queries.', '2024-02-15', 'Improved application performance with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('WE6587', 'Upgrade the MySQL database to the latest version to support new features and improve performance.', '2025-03-01', 'Improved database performance with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('IO3245', 'Implement a disaster recovery plan to ensure business continuity in the event of a disaster.', '2024-03-01', 'Improved disaster recovery with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('AS4356', 'Migrate the application to a cloud-based infrastructure to improve scalability and reduce costs.', '2025-01-15', 'Improved scalability with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('KL7890', 'Introduce a security information and event management (SIEM) system to improve security monitoring and incident response.', '2024-02-15', 'Improved security with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('PO3218', 'Upgrade the network infrastructure to support faster data transfer rates and improve overall network performance.', '2025-03-01', 'Improved network performance with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('YU7654', 'Implement a data backup and recovery system to ensure data integrity and availability.', '2024-03-01', 'Improved data integrity with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('BN8765', 'Migrate the application to a containerized infrastructure to improve scalability and reduce costs.', '2025-01-15', 'Improved scalability with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('VC5432', 'Introduce a monitoring system to track application performance and identify areas for improvement.', '2024-02-15', 'Improved application performance with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('CX9876', 'Upgrade the storage infrastructure to support faster data access and improve overall system performance.', '2025-03-01', 'Improved system performance with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('ZA1111', 'Implement a vulnerability management system to identify and remediate security vulnerabilities.', '2024-03-01', 'Improved security with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('XV2222', 'Migrate the database to a cloud-based solution to improve scalability and reduce costs.', '2025-01-15', 'Improved scalability with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('WE3333', 'Introduce a compliance management system to ensure regulatory compliance and reduce risk.', '2024-02-15', 'Improved compliance with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('IO4444', 'Upgrade the network security infrastructure to support improved threat detection and incident response.', '2025-03-01', 'Improved security with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('PO5555', 'Migrate the application to a serverless infrastructure to improve scalability and reduce costs.', '2024-03-01', 'Improved scalability with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('ZA6666', 'Implement a continuous integration and continuous delivery (CI/CD) pipeline to improve application delivery speed and quality.', '2025-01-15', 'Improved application delivery speed with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('BN7777', 'Introduce a data analytics platform to improve business insights and decision-making.', '2024-02-15', 'Improved business insights with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('GH8888', 'Upgrade the IT service management system to improve incident management and problem management.', '2025-03-01', 'Improved incident management with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('VC9999', 'Migrate the database to a NoSQL database to improve scalability and performance.', '2024-03-01', 'Improved scalability with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('CX1234', 'Implement a cloud security gateway to improve security and compliance in the cloud.', '2025-01-15', 'Improved security with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('XV2345', 'Introduce a digital asset management system to improve asset management and reduce risk.', '2024-02-15', 'Improved asset management with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('WE3456', 'Upgrade the identity and access management system to improve security and compliance.', '2025-03-01', 'Improved security with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('IO4567', 'Migrate the application to a microservices architecture to improve scalability and reduce costs.', '2024-03-01', 'Improved scalability with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('PO5678', 'Implement a business continuity management system to improve business continuity and reduce risk.', '2025-01-15', 'Improved business continuity with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('ZA6789', 'Introduce a threat intelligence platform to improve threat detection and incident response.', '2024-02-15', 'Improved security with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('BN7890', 'Upgrade the security orchestration, automation, and response (SOAR) system to improve security incident response.', '2025-03-01', 'Improved security incident response with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('VC8765', 'Migrate the database to a graph database to improve data relationships and insights.', '2024-03-01', 'Improved data insights with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('CX9876', 'Implement a data loss prevention (DLP) system to improve data security and compliance.', '2025-01-15', 'Improved data security with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('XV0987', 'Upgrade the cloud management platform to improve cloud management and reduce costs.', '2024-02-15', 'Improved cloud management with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('WE1098', 'Migrate the application to a serverless architecture to improve scalability and reduce costs.', '2025-03-01', 'Improved scalability with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('IO2109', 'Implement a security awareness training program to improve security awareness and reduce risk.', '2024-03-01', 'Improved security awareness with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('PO3210', 'Upgrade the IT asset management system to improve asset management and reduce risk.', '2025-01-15', 'Improved asset management with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('ZA4321', 'Introduce a vendor risk management system to improve vendor risk management and reduce risk.', '2024-02-15', 'Improved vendor risk management with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('BN5432', 'Migrate the database to a document-oriented database to improve data management and insights.', '2025-03-01', 'Improved data insights with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('VC6543', 'Implement a continuous monitoring system to improve security and compliance.', '2024-03-01', 'Improved security with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('CX7654', 'Upgrade the network management system to improve network management and reduce costs.', '2025-01-15', 'Improved network management with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('XV8765', 'Migrate the application to a containerized architecture to improve scalability and reduce costs.', '2024-02-15', 'Improved scalability with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('WE9876', 'Implement a security incident response plan to improve security incident response.', '2025-03-01', 'Improved security incident response with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('IO1098', 'Upgrade the identity and access management system to improve security and compliance.', '2024-03-01', 'Improved security with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('PO2109', 'Migrate the database to a relational database to improve data relationships and insights.', '2025-01-15', 'Improved data insights with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('ZA3210', 'Implement a business continuity plan to improve business continuity and reduce risk.', '2024-02-15', 'Improved business continuity with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('BN4321', 'Upgrade the security awareness training program to improve security awareness and reduce risk.', '2025-03-01', 'Improved security awareness with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('VC5432', 'Migrate the application to a microservices architecture to improve scalability and reduce costs.', '2024-03-01', 'Improved scalability with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('CX6543', 'Implement a continuous integration and continuous delivery (CI/CD) pipeline to improve application delivery speed and quality.', '2025-01-15', 'Improved application delivery speed with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('XV7654', 'Upgrade the IT asset management system to improve asset management and reduce risk.', '2024-02-15', 'Improved asset management with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('WE8765', 'Migrate the database to a NoSQL database to improve scalability and performance.', '2025-03-01', 'Improved scalability with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('IO9876', 'Implement a security information and event management (SIEM) system to improve security monitoring and incident response.', '2024-03-01', 'Improved security with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('PO1098', 'Upgrade the network infrastructure to support faster data transfer rates and improve overall network performance.', '2025-01-15', 'Improved network performance with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('ZA2109', 'Migrate the application to a serverless architecture to improve scalability and reduce costs.', '2024-02-15', 'Improved scalability with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned');

In [ ]:
INSERT INTO GEN_AI_FSI.DTCC_HACKATHON.CHANGE_REQUEST_RAW (ChangeNumber, Description, Date, Impact, Priority, Risk, Justification, State, Disposition, Category) VALUES ('9A3512', 'Upgrade the Redis cache server to version 6.2.5 to improve cache hit ratio and reduce latency.', '2025-02-15', 'Improved performance with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('1F2468', 'Modify the existing firewall rules to allow incoming traffic on port 443 for the new HTTPS endpoint.', '2025-03-01', 'Improved security with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('7C9821', 'Implement a new messaging queue system using Apache Kafka to improve message processing throughput.', '2025-02-10', 'Improved performance with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('5D4937', 'Update the existing SSL certificates to use SHA-256 encryption algorithm for improved security.', '2025-02-12', 'Improved security with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('2E6551', 'Reconfigure the existing load balancer to use a weighted round-robin algorithm for improved traffic distribution.', '2025-02-18', 'Improved system reliability with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('3B2198', 'Implement a new monitoring system using Prometheus and Grafana to improve system visibility and alerting.', '2025-02-22', 'Improved system reliability with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('1C6714', 'Upgrade the existing Node.js application to version 14.17.0 to improve performance and security.', '2025-02-20', 'Improved performance with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('9E8225', 'Modify the existing database schema to add new columns for improved data analytics.', '2025-02-25', 'Improved data analytics with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('7F4673', 'Implement a new backup and disaster recovery system using AWS S3 and Glacier for improved data protection.', '2025-02-28', 'Improved data protection with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('5G9832', 'Update the existing Jenkins CI/CD pipeline to use Docker containers for improved build and deployment efficiency.', '2025-03-05', 'Improved build and deployment efficiency with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('3H1357', 'Reconfigure the existing network switches to use VLANs for improved network segmentation and security.', '2025-03-08', 'Improved network security with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('1J6492', 'Implement a new security information and event management (SIEM) system using Splunk for improved security visibility and alerting.', '2025-03-10', 'Improved security visibility and alerting with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('9K9821', 'Upgrade the existing Python application to version 3.9.5 to improve performance and security.', '2025-03-12', 'Improved performance with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('7L4673', 'Modify the existing firewall rules to allow incoming traffic on port 80 for the new HTTP endpoint.', '2025-03-15', 'Improved security with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('5M9832', 'Implement a new automation system using Ansible for improved deployment efficiency and consistency.', '2025-03-18', 'Improved deployment efficiency and consistency with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('3N1357', 'Update the existing SSL certificates to use elliptic curve cryptography (ECC) for improved security.', '2025-03-20', 'Improved security with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('1P6492', 'Reconfigure the existing database to use a master-slave replication for improved data redundancy and availability.', '2025-03-22', 'Improved data redundancy and availability with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('9Q9821', 'Implement a new network access control (NAC) system using Cisco ISE for improved network security.', '2025-03-25', 'Improved network security with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('7R4673', 'Upgrade the existing Java application to version 11.0.10 to improve performance and security.', '2025-03-28', 'Improved performance with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('5S9832', 'Modify the existing load balancer to use a least connections algorithm for improved traffic distribution.', '2025-04-01', 'Improved system reliability with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('3T1357', 'Implement a new virtual private network (VPN) system using OpenVPN for improved remote access security.', '2025-04-03', 'Improved remote access security with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('1U6492', 'Update the existing antivirus software to use machine learning-based detection for improved malware detection.', '2025-04-05', 'Improved malware detection with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('9V9821', 'Reconfigure the existing network to use a virtual local area network (VLAN) for improved network segmentation and security.', '2025-04-08', 'Improved network security with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('7W4673', 'Implement a new web application firewall (WAF) system using AWS WAF for improved web application security.', '2025-04-10', 'Improved web application security with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('5X9832', 'Upgrade the existing C++ application to version 14.3.2 to improve performance and security.', '2025-04-12', 'Improved performance with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('3Y1357', 'Modify the existing database schema to add new tables for improved data analytics.', '2025-04-15', 'Improved data analytics with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('1Z6492', 'Implement a new threat intelligence system using ThreatQuotient for improved threat detection and response.', '2025-04-18', 'Improved threat detection and response with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('9A9821', 'Reconfigure the existing network to use a software-defined wide area network (SD-WAN) for improved network connectivity and security.', '2025-04-22', 'Improved network connectivity and security with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('7B4673', 'Upgrade the existing Ruby application to version 3.0.1 to improve performance and security.', '2025-04-25', 'Improved performance with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'), ('5C9832', 'Implement a new continuous monitoring system using Elasticsearch for improved system visibility and alerting.', '2025-04-28', 'Improved system visibility and alerting with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('3D1357', 'Update the existing intrusion detection system (IDS) to use machine learning-based detection for improved threat detection.', '2025-05-01', 'Improved threat detection with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('1E6492', 'Reconfigure the existing network to use a virtual private cloud (VPC) for improved network segmentation and security.', '2025-05-03', 'Improved network security with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'), ('9F9821', 'Implement a new security orchestration, automation, and response (SOAR) system using Splunk Phantom for improved security incident response.', '2025-05-06', 'Improved security incident response with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'), ('7G4673', 'Upgrade the existing .NET application to version 5.0.4 to improve performance and security.', '2025-05-08', 'Improved performance with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned');

In [ ]:
def generate_risk_prediction_prompt(cr_data):
    prompt = f"""
            <role>
            You are an experienced dev ops professional deeply knowledgeable on computer systems that support a very large company and the metadata that is captured about change requests.
            A change request is a formal proposal for an alteration to the computer system that you manage.
            As a dev ops expert, you specialize in using the metadata provided about a change request to predict the liklihood of the change request unintentionally destabalizing the computer system.
            You are going to be provided with change request meta data as a json object held between <cr_data> and your job is to provide a prediction score and reasoning behind the risk score in the <output> section. 
            </role>
        
            <task>: Follow these instructions,
            1) Considering the <cr_data> and your <role>, provide a risk score between 0 to 5 of this change request destabalizing the computer system when deployed. Output this as [Risk_Score]. Then,
            2) Considering the <cr_data> and your <role>, provide a reasoning for the risk score in as few words as possible while maintaining all detail needed to understand your reasoning. Output this as [Risk_Score_Reason]
            </task>

            <cr_data>
            {cr_data}
            </cr_data>
        
            <Output> 
            produce valid JSON. Absolutely do not include any additional text before or following the JSON. Output should use following <JSON_format>
            </Output>
            
            <JSON_format>
            {{
                "Risk_Score": (A risk score between 0 to 5 of this change request destabalizing the computer system when deployed),
                "Risk_Score_Reason": (A concise resoning for the Risk_Score),
            }}
            </JSON_format>
            """
    return prompt

In [ ]:
import streamlit as st
from snowflake.cortex import Complete
from snowflake.snowpark.functions import col, call_udf
from snowflake.snowpark.context import get_active_session

#get a session
session = get_active_session()

#replace this with your table
database = 'GEN_AI'
schema = 'PUBLIC'
table = 'ChangeRequests'

#get the data into pandas
#this could be any data but I am using summaries of state of the unions-- replace this with whatever you want.
df = session.table(f"{database}.{schema}.{table}").to_pandas()

#each row's columns are collapsed into a json object. that is easier to pass to the model
#NOTICE: I am calling my generate_prompt function and passing my row as json.
#NOTICE that I am calling complete function here (https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex)
    # VALUE CALLOUT: I could switch out any LLM I have access to (https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions#availability)
df['RISK_ASSESSMENT'] = df.apply(lambda row: Complete('mistral-large2', generate_risk_prediction_prompt(row.to_json())), axis = 1)

#!!look into call_udf amd call function for further efficiency gain via parallelisation!!
    # call_udf: https://docs.snowflake.com/en/developer-guide/snowpark/python/calling-functions#calling-user-defined-functions-udfs
    # call_function: https://docs.snowflake.com/en/developer-guide/snowpark/python/calling-functions#calling-system-defined-functions

#show us the data
st.dataframe(df['RISK_ASSESSMENT'])